In [1]:
# First steps for intake-ESGF-Catalog (https://intake-esgf.readthedocs.io/en/latest/quickstart.html)
from intake_esgf import ESGFCatalog
import numpy as np
import xarray as xr
import netCDF4


In [2]:
# Defined all functions used in this script

# Placeholder function to generate a sample of weights
def generate_sample_of_weights(num_models):
    """
    Generate a sample of weights that are non-negative and sum up to 1.
    Args:
    num_models (int): The number of models (and hence the number of weights to generate)
    Returns:
    numpy.ndarray: An array of weights that sum up to 1.
    """
    # Generate uniform samples which naturally fall between 0 and 1
    weights = np.random.uniform(0, 1, size=num_models)
    weights = weights **4
    # Normalize the weights so they sum up to 1
    normalized_weights = weights / np.sum(weights)
    return normalized_weights

# Placeholder for 'models' which would be an array of models.
# Here we assume each model is a function that predicts an output given an input.
# In the actual use case, these should be replaced with real predictive models.
def model_predictions(models_1, models_2, models_3, models_4, models_5, models_6, models_7, models_8, models_9, models_10, weights_norm):
    model_mean_weighted = (weights_norm[0]*models_1) + (weights_norm[1]*models_2) + (weights_norm[2]*models_3) + (weights_norm[3]* models_4) + (weights_norm[4]*models_5) + (weights_norm[5]*models_6) + (weights_norm[6]*models_7) + (weights_norm[7]*models_8) + (weights_norm[8]*models_9) + (weights_norm[9]*models_10)
    return model_mean_weighted

# function for shifiting lon according to Bharat's code
def shift_lon(model):
    # First shift the lon dimension for the model average
    ds_tmp = model.copy(deep=True)
    ds_tmp['lon'] = ds_tmp['lon'] - 180

    gpp_reshape = np.zeros(ds_tmp.shape)
    gpp_reshape[:,ds_tmp['lon'].size//2:] = ds_tmp[:,:ds_tmp['lon'].size//2].data
    gpp_reshape[:,:ds_tmp['lon'].size//2] = ds_tmp[:,ds_tmp['lon'].size//2:].data
    ds_tmp.data = gpp_reshape
    return ds_tmp 

# Create function that will do RMSE for each seperate model

def get_RMSE(model, obs):
    # Step 2: Compute the Difference
    difference = model - obs
    # Step 3: Square the Difference
    squared_difference = difference ** 2
    # Step 4: Compute the Mean Squared Error
    mse = squared_difference.mean(dim=['lat', 'lon'])
    # Step 5: Take the Square Root to get RMSE
    rmse  = np.sqrt(mse)
    rmse_weighted = rmse.values.item()

    return rmse_weighted

In [3]:
#Populate the Catalog - bringing in nothing from the catalog
cat = ESGFCatalog()
print(cat)  # <-- nothing to see here yet

Perform a search() to populate the catalog.


In [4]:
# Import selected models from Intake-ESGF Catalog for selected variable

models = ["ACCESS-ESM1-5","IPSL-CM6A-LR","CESM2", "UKESM1-0-LL","BCC-CSM2-MR","MPI-ESM1-2-HR","CanESM5","GFDL-ESM4","NorESM2-LM", "MIROC-ES2L"]

cat.search(
    experiment_id="historical",
    source_id= models,
    frequency="mon",
    variable_id=["gpp"],
)
cat.remove_ensembles()

   Searching indices:   0%|          |0/2 [       ?index/s]

Summary information for 10 results:
mip_era                                                     [CMIP6]
experiment_id                                          [historical]
variable_id                                                   [gpp]
grid_label                                            [gr1, gn, gr]
institution_id    [NOAA-GFDL, NCC, IPSL, CSIRO, NCAR, BCC, MOHC,...
member_id                                      [r1i1p1f1, r1i1p1f2]
table_id                                                     [Lmon]
source_id         [GFDL-ESM4, NorESM2-LM, IPSL-CM6A-LR, ACCESS-E...
activity_drs                                                 [CMIP]
project                                                     [CMIP6]
dtype: object

In [5]:
#Obtaining the datasets and loading it into a dictionary (putting it in the shopping cart)

dsd = cat.to_dataset_dict()

Get file information:   0%|          |0/2 [       ?index/s]

Adding cell measures:   0%|          |0/10 [     ?dataset/s]

In [6]:
#printing variable keys to see how variable names are set up
print(dsd.keys())

dict_keys(['gn.NCC.r1i1p1f1.NorESM2-LM', 'gn.MPI-M.r1i1p1f1.MPI-ESM1-2-HR', 'gr.IPSL.r1i1p1f1.IPSL-CM6A-LR', 'gn.MIROC.r1i1p1f2.MIROC-ES2L', 'gn.NCAR.r1i1p1f1.CESM2', 'gn.CCCma.r1i1p1f1.CanESM5', 'gn.MOHC.r1i1p1f2.UKESM1-0-LL', 'gr1.NOAA-GFDL.r1i1p1f1.GFDL-ESM4', 'gn.BCC.r1i1p1f1.BCC-CSM2-MR', 'gn.CSIRO.r1i1p1f1.ACCESS-ESM1-5'])


In [15]:
# # Define models for selected time periods and fixed units (make sure that model names match the printed dict_key (above))
# list_keys = (list(dsd.keys()))

# model_1 = dsd[list_keys[0]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
# model_2 = dsd[list_keys[1]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
# model_3 = dsd[list_keys[2]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
# model_4 = dsd[list_keys[3]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
# model_5 = dsd[list_keys[4]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
# model_6 = dsd[list_keys[5]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
# model_7 = dsd[list_keys[6]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
# model_8 = dsd[list_keys[7]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
# model_9 = dsd[list_keys[8]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
# model_10 = dsd[list_keys[9]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000

# # Find out which model is CanESM (lowest resolution fo regridding)
# for idx, item in enumerate(list_keys):
#     if 'CanESM' in item:
#         print(idx, item)
#         break

# model_CanESM_id = idx
# print(model_CanESM_id)

5 gn.CCCma.r1i1p1f1.CanESM5
5


In [20]:
# Load each model in its own dataset

# list keys of model names
list_keys = (list(dsd.keys()))

# Find out which model to load weights into
for idx, item in enumerate(list_keys):
    if 'ACCESS' in item:
        model_1 = dsd[list_keys[idx]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
    if 'BCC' in item:
        model_2 = dsd[list_keys[idx]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
    if 'CanESM5' in item:
        model_3 = dsd[list_keys[idx]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
    if 'CESM2' in item:
        model_4 = dsd[list_keys[idx]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
    if 'GFDL' in item:
        model_5 = dsd[list_keys[idx]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
    if 'IPSL' in item:
        model_6 = dsd[list_keys[idx]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
    if 'MIROC' in item:
        model_7 = dsd[list_keys[idx]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
    if 'MPI' in item:
        model_8 = dsd[list_keys[idx]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
    if 'NorESM2' in item:
        model_9 = dsd[list_keys[idx]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000
    if 'UKESM1' in item:
        model_10 = dsd[list_keys[idx]]["gpp"].sel(time=slice('1980-01-01', '2013-12-01'))* 86400 * 1000 # .mean(dim="time") * 86400 * 1000

# Find out which model is CanESM (lowest resolution fo regridding)
for idx, item in enumerate(list_keys):
    if 'CanESM' in item:
        print(idx, item)
        break

model_CanESM_id = idx
print(model_CanESM_id)

5 gn.CCCma.r1i1p1f1.CanESM5
5


In [21]:
# Linear Interpolation/Regridding to match lowest resolution model (CanESM5):

#Extract lat/lon grid from CanESM5
lat_target = dsd[list_keys[idx]]["gpp"].lat.values
lon_target = dsd[list_keys[idx]]["gpp"].lon.values

# Regrid each model and take long (not longtitude) term mean 
gpp_model_1_Regridded = model_1.interp(lat=lat_target, lon=lon_target)
gpp_model_1_Regridded_mean = gpp_model_1_Regridded.mean(dim="time")

gpp_model_2_Regridded = model_2.interp(lat=lat_target, lon=lon_target)
gpp_model_2_Regridded_mean = gpp_model_2_Regridded.mean(dim="time")

gpp_model_3_Regridded = model_3.interp(lat=lat_target, lon=lon_target)
gpp_model_3_Regridded_mean = gpp_model_3_Regridded.mean(dim="time")

gpp_model_4_Regridded = model_4.interp(lat=lat_target, lon=lon_target)
gpp_model_4_Regridded_mean = gpp_model_4_Regridded.mean(dim="time")

gpp_model_5_Regridded = model_5.interp(lat=lat_target, lon=lon_target)
gpp_model_5_Regridded_mean = gpp_model_5_Regridded.mean(dim="time")

gpp_model_6_Regridded = model_6.interp(lat=lat_target, lon=lon_target)
gpp_model_6_Regridded_mean = gpp_model_6_Regridded.mean(dim="time")

gpp_model_7_Regridded = model_7.interp(lat=lat_target, lon=lon_target)
gpp_model_7_Regridded_mean = gpp_model_7_Regridded.mean(dim="time")

gpp_model_8_Regridded = model_8.interp(lat=lat_target, lon=lon_target)
gpp_model_8_Regridded_mean = gpp_model_8_Regridded.mean(dim="time")

gpp_model_9_Regridded = model_9.interp(lat=lat_target, lon=lon_target)
gpp_model_9_Regridded_mean = gpp_model_9_Regridded.mean(dim="time")

gpp_model_10_Regridded = model_10.interp(lat=lat_target, lon=lon_target)
gpp_model_10_Regridded_mean = gpp_model_10_Regridded.mean(dim="time")

In [22]:
# Shift every model to proper coordinates
gpp_model_1_Regridded_mean_shifted = shift_lon(gpp_model_1_Regridded_mean)
gpp_model_2_Regridded_mean_shifted = shift_lon(gpp_model_2_Regridded_mean)
gpp_model_3_Regridded_mean_shifted = shift_lon(gpp_model_3_Regridded_mean)
gpp_model_4_Regridded_mean_shifted = shift_lon(gpp_model_4_Regridded_mean)
gpp_model_5_Regridded_mean_shifted = shift_lon(gpp_model_5_Regridded_mean)
gpp_model_6_Regridded_mean_shifted = shift_lon(gpp_model_6_Regridded_mean)
gpp_model_7_Regridded_mean_shifted = shift_lon(gpp_model_7_Regridded_mean)
gpp_model_8_Regridded_mean_shifted = shift_lon(gpp_model_8_Regridded_mean)
gpp_model_9_Regridded_mean_shifted = shift_lon(gpp_model_9_Regridded_mean)
gpp_model_10_Regridded_mean_shifted = shift_lon(gpp_model_10_Regridded_mean)

In [23]:
# Define the observation 

# Open the NetCDF file as an xarray Dataset
ds = xr.open_dataset('/Users/6i0/Documents/Data/gpp_WECANN.nc')

# Access the GPP variable from the Dataset
gpp_data = ds['gpp']
gpp_data_mean = gpp_data.mean(dim="time")
lat = ds['lat']
lon = ds['lon']
time = ds['time']

#gpp_data_Regridded_mean = gpp_data_mean
lat_target = dsd[list_keys[idx]]["gpp"].lat.values
lon_target = dsd[list_keys[idx]]["gpp"].lon.values -180

# Correctly using ds_emean for interpolation
gpp_data_Regridded = gpp_data.interp(lat=lat_target, lon=lon_target)
gpp_data_Regridded_mean = gpp_data_Regridded.mean(dim="time")

#Shift data
gpp_data_Regridded_mean_shifted = shift_lon(gpp_data_Regridded_mean)


In [26]:
# BMA Implementation:

# Placeholder for actual observed data (data_obs)
data_obs = gpp_data_Regridded_mean

# Number of models we are combining
num_models = 10 
# Number of BMA Samples 
n_samples= 10000

# Initialize arrays to hold weights, weighted averages and RMSE
new_weights = np.zeros((n_samples, num_models))
weighted_avgs = np.zeros((n_samples, len(data_obs)))
rmse_weighted = np.zeros(n_samples)

# Loop to generate weights, calculate weighted averages and RMSE
for i in range(n_samples):
    weights = generate_sample_of_weights(num_models)
    new_weights[i, :] = weights
    # For this example, we assume that all models have a single input data point, for simplicity.
    # In a real scenario, the input to models would be the data they should predict.
    weighted_avgs = model_predictions(gpp_model_1_Regridded_mean_shifted, gpp_model_2_Regridded_mean_shifted, gpp_model_3_Regridded_mean_shifted, gpp_model_4_Regridded_mean_shifted, gpp_model_5_Regridded_mean_shifted, gpp_model_6_Regridded_mean_shifted, gpp_model_7_Regridded_mean_shifted, gpp_model_8_Regridded_mean_shifted, gpp_model_9_Regridded_mean_shifted, gpp_model_10_Regridded_mean_shifted, new_weights[i, :])
    
    #Estimate RMSE for this BMA samples
    rmse_weighted[i] = get_RMSE(weighted_avgs, data_obs) 

# Sorting RMSE to find the best one 
sorted_indices = np.argsort(rmse_weighted)
best_rmse = rmse_weighted[sorted_indices[0]]
best_location = sorted_indices[0]
best_weights = new_weights[best_location, :]

# Print to screen the best RMSE, Location of that sample, and the corresponding weights
print(list_keys)
best_rmse, best_location, best_weights


['gn.NCC.r1i1p1f1.NorESM2-LM', 'gn.MPI-M.r1i1p1f1.MPI-ESM1-2-HR', 'gr.IPSL.r1i1p1f1.IPSL-CM6A-LR', 'gn.MIROC.r1i1p1f2.MIROC-ES2L', 'gn.NCAR.r1i1p1f1.CESM2', 'gn.CCCma.r1i1p1f1.CanESM5', 'gn.MOHC.r1i1p1f2.UKESM1-0-LL', 'gr1.NOAA-GFDL.r1i1p1f1.GFDL-ESM4', 'gn.BCC.r1i1p1f1.BCC-CSM2-MR', 'gn.CSIRO.r1i1p1f1.ACCESS-ESM1-5']


(0.5536644728234702,
 5677,
 array([0.04999625, 0.00140119, 0.18983852, 0.07555226, 0.12824551,
        0.22720795, 0.17796318, 0.10856709, 0.04043152, 0.00079654]))